# Tarea Del Discurso.

**Integrantes**: Nicolás Cenzano, Juan Araya y Andrea Engdahl.

**Observación** Los archivos deben estar en codificación utf-8.

In [ ]:
import os
import numpy as np
import es_core_news_sm
import regex
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
import joblib
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from scipy.spatial.distance import cosine
import random

In [ ]:
# En caso de pedir o tener problema con los puntos, es necesario correr la siguiente línea.
#nltk.download('punkt') #Para los puntos

## Funciones.

Se usan varias de las funciones realizadas por el profesor.

In [ ]:
def CrearEspacioLSA(corpus,numDim,NombreModelo):
  textos = PreProcesar(corpus)
  transf = TfidfVectorizer()
  tf = transf.fit_transform(textos).T
  U, Sigma, VT = np.linalg.svd(tf.toarray())
  terms = np.dot(U[:,:numDim], np.diag(Sigma[:numDim]))
  docs = np.dot(np.diag(Sigma[:numDim]), VT[:numDim, :]).T 
  vocab = transf.get_feature_names()
  GrabarModeloLSA(NombreModelo, Sigma, terms, docs, vocab)
  #return(textos)

def GrabarModeloLSA(NombreModelo,Sigma,terms,docs, vocab):
   existe = os.path.isdir(NombreModelo)
   if not existe:
       os.mkdir(NombreModelo)
   joblib.dump(Sigma,   NombreModelo +"/"+'sigma.pkl') 
   joblib.dump(terms,   NombreModelo +"/"+'terms.pkl') 
   joblib.dump(docs,    NombreModelo +"/"+'docs.pkl') 
   joblib.dump(vocab,   NombreModelo +"/"+'vocab.pkl') 

    
def CargarModeloLSA(NombreModelo):
    sigma   = joblib.load(NombreModelo+"/"+'sigma.pkl')
    terms   = joblib.load(NombreModelo+"/"+'terms.pkl')
    docs    = joblib.load(NombreModelo+"/"+'docs.pkl')
    vocab   = joblib.load(NombreModelo+"/"+'vocab.pkl')
    return(sigma, terms, docs, vocab)

def CrearDiccionario(Vectores,vocabulario):
   dicc = {}
   for  v in range(0,len(vocabulario)):
      dicc[vocabulario[v]] = Vectores[v]
   return(dicc)


def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="utf-8").read()
     corpus.append(texto)
  return(corpus,directorio)

def PreProcesar(textos):
    texto_limpio = []
    for texto in textos:  
        texto = EliminaNumeroYPuntuacion(texto)
        texto = EliminarSaltosLinea(texto)
        texto = EliminarEspaciosExtras(texto)
        #texto = Lematizar(texto)
        texto_limpio.append(texto)
    return(texto_limpio)

# Es necesario borrar símbolos que no estaban previamente identificados en puctuation
def EliminaNumeroYPuntuacion(oracion):
    string_numeros = regex.sub(r'[-\.\”\“\¿\?\!\¡\°\d+\…\–\´]','', oracion) 
    return(''.join(c for c in string_numeros if c not in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿¡°\”\“\…\–\´')) #punctuation
           
def EliminarSaltosLinea(oracion):
    string = regex.sub(r'[\n\r\t\xa0\xad]',' ', oracion)
    return(''.join(c for c in string))

def EliminarEspaciosExtras(oracion):
    string = regex.sub(r'\s{2,}',' ', oracion)
    return(''.join(c for c in string))


def Similitud(Vec1,Vec2):
    sim = 1 - cosine(Vec1, Vec2)
    return(sim)

def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))


## Crear el LSA y El Corpus.

In [ ]:
PATH="Discursos\\"

In [ ]:
nlp    = es_core_news_sm.load()
NumDim =  12
corpus,directorio = CrearCorpus(PATH)

CrearEspacioLSA(corpus,NumDim,"mi_lsa")

## Cargar el modelo LSA 

In [ ]:
(Sigma, vect_terms, vect_docs, lista_palabras) = CargarModeloLSA("mi_lsa")

In [ ]:
terms = CrearDiccionario(vect_terms, lista_palabras)

In [ ]:
#Número de líneas del documento a generar
N=20
#selección aleatoria del primer documento.
primer_documento=random.randint(0, len(corpus))

# Añade la primera oración
Texto=[sent_tokenize(corpus[primer_documento], language='spanish')[0]]


In [ ]:

for i in range(0,N):
    #inicialización y penultima oración.
    
    # Esta parte es para considerar el número de frases que tiene el documento, 
    # para su aplicación en el cálculo del promedio
    if i==0:
        n_oracion=len(sent_tokenize(corpus[primer_documento], language='spanish'))
    else:
        n_oracion=len(Nuevo_Documento)
        
    #Seleccionamos la primera línea o la última añadida para compararla con la nueva a ingresar.
    # se considera el promedio como la suma del vector de palabras de la oración, 
    # dividido por el número de oraciones detectadas en el documento.
    
    arregloO=word_tokenize(PreProcesar(Texto)[i].lower(), language='spanish')
    #print(PreProcesar(Texto)[i].lower())
    vector_oracionO=sum([terms[i] for i in arregloO if len(i)>1])/n_oracion 
    
    # Elige el nuevo documento, cuya oración se podría añadir por similitud
    nd=random.randint(0, len(corpus))
    Nuevo_Documento=PreProcesar(sent_tokenize(corpus[nd], language='spanish'))


    # Se recorre línea a línea del documento y se separan en sus palabras
    valor=0
    for j in range(0,len(Nuevo_Documento)):
        linea=word_tokenize(Nuevo_Documento[j].lower(), language='spanish')
        vector_oracion=sum([terms[i] for i in linea if len(i)>1])/len(Nuevo_Documento) #calcula el promedio
        
        similaridad=Similitud(vector_oracionO,vector_oracion)
        
        if (valor <= similaridad) & (similaridad!=1.0) : 
            #se descarta la similaridad ==1, pues en ese caso sería la misma oración
            # nos quedamos con la mayor similitud
            print(j)
            print(similaridad)
            print(Nuevo_Documento[j])
            valor=similaridad
            nueva_oracion=sent_tokenize(corpus[nd], language='spanish')[j] # se añade la oración de mayor similitud, pero se toma la original sin preprocesamiento.
    
    Texto.append(nueva_oracion)

    

## Se guarda el discurso generado.

In [ ]:
file = open('texto_final.txt', "w")
file.write(' '.join(regex.sub(r'[\n\r\t\xa0\xad]',' ', linea) for linea in Texto))
file.close()